# 06 · Heatmaps: 2021 / 2022 / 2024 + Stichprobe 2025Zwei Heatmaps:- **Werte-Heatmap** der Anteile in % pro Kategorie und Jahr- **Δ-Heatmap** (Stichprobe 2025 − Statista 2024) in **Prozentpunkten (pp)**Hinweis: Mehrfachnennungen möglich → Werte können > 100 % sein.

In [3]:
# 06_heatmap_rows — Cell 1: Setup & Daten laden
from __future__ import annotations
from pathlib import Path
import json
import pandas as pd

BASE = Path.cwd().resolve().parents[0] if Path.cwd().name.lower()=="notebooks" else Path.cwd()
OUT  = BASE / "data" / "processed"
FIG  = BASE / "reports" / "figures"
TAB  = BASE / "reports" / "tables"
FIG.mkdir(parents=True, exist_ok=True)
TAB.mkdir(parents=True, exist_ok=True)

CONFIG = json.loads((OUT / "project_config.json").read_text(encoding="utf-8"))
KANON  = CONFIG["kanon"]

# Datensätze laden
stat_wide   = pd.read_csv(OUT / "statista_harmonisiert_2021_2022_2024.csv").set_index("Kategorie").reindex(KANON).fillna(0.0)
sample_wide = pd.read_csv(OUT / "umfrage_2025_wide.csv").set_index("Kategorie").reindex(KANON).fillna(0.0)

def _to_int_if_year(c):
    s = str(c).strip()
    if s.endswith(".0") and s[:-2].isdigit():
        s = s[:-2]
    return int(s) if s.isdigit() else s

stat_wide.columns   = [_to_int_if_year(c) for c in stat_wide.columns]
sample_wide.columns = [_to_int_if_year(c) for c in sample_wide.columns]

YEARS = [y for y in [2021, 2022, 2024] if y in stat_wide.columns]
assert YEARS, "Keine Statista-Jahre gefunden."
assert 2025 in sample_wide.columns, "Stichprobe 2025 fehlt."

# Kombinierte Matrix für die Heatmap (Spaltenreihenfolge: 2021, 2022, 2024, Stichprobe 2025)
values = stat_wide[YEARS].copy()
values["Stichprobe 2025"] = sample_wide[2025]

# Optional sortieren (nach 2024 aufsteigend, wenn vorhanden)
order_by = 2024 if 2024 in values.columns else YEARS[-1]
values = values.sort_values(order_by, ascending=True)

display(values.head(3))


,2021,2022,2024,Stichprobe 2025
Kategorie,,,,
Möbel / Wohnaccessoires,38.0,32.0,15.0,0.000000
Lebensmittel / Getränke,33.0,28.0,16.0,15.384615
Bücher / Medien / Software,0.0,92.0,25.0,46.153846


In [4]:
# 06_heatmap_rows — Cell 2: Δ 2025–2024 (pp)
if 2024 in values.columns:
    delta = (values["Stichprobe 2025"] - values[2024]).to_frame(name="Δ pp 2025–2024")
else:
    delta = pd.DataFrame(index=values.index, data={"Δ pp 2025–2024": float("nan")})

display(delta.head(3))


,Δ pp 2025–2024
Kategorie,
Möbel / Wohnaccessoires,-15.000000
Lebensmittel / Getränke,-0.615385
Bücher / Medien / Software,21.153846


In [5]:
# 06_heatmap_rows — Cell 3: Pro Kategorie eine Zeile (Werte)
lines_values = []
cols = ["2021", "2022", "2024", "Stichprobe 2025"]
# sichere Spaltenliste basierend auf real vorhandenen Spalten
present_cols = [c for c in [2021, 2022, 2024] if c in values.columns] + ["Stichprobe 2025"]

for cat, row in values.iterrows():
    parts = [f"{int(c)}={row[c]:.1f} %" if isinstance(c, int) else f"{c}={row[c]:.1f} %"
             for c in present_cols]
    lines_values.append(f"{cat}: " + " | ".join(parts))

print("\n".join(lines_values))


Möbel / Wohnaccessoires: 2021=38.0 % | 2022=32.0 % | 2024=15.0 % | Stichprobe 2025=0.0 %
Lebensmittel / Getränke: 2021=33.0 % | 2022=28.0 % | 2024=16.0 % | Stichprobe 2025=15.4 %
Bücher / Medien / Software: 2021=0.0 % | 2022=92.0 % | 2024=25.0 % | Stichprobe 2025=46.2 %
Elektronik (z. B. Smartphones, Haushaltsgeräte): 2021=38.0 % | 2022=145.0 % | 2024=38.0 % | Stichprobe 2025=43.6 %
Medikamente / Drogerieartikel: 2021=78.0 % | 2022=92.0 % | 2024=51.0 % | Stichprobe 2025=28.2 %
Hobby- & Freizeitartikel: 2021=83.0 % | 2022=71.0 % | 2024=57.0 % | Stichprobe 2025=53.8 %
Kleidung / Schuhe: 2021=57.0 % | 2022=78.0 % | 2024=106.0 % | Stichprobe 2025=69.2 %


In [6]:
# 06_heatmap_rows — Cell 4: Pro Kategorie eine Zeile (Δ 2025–2024)
lines_delta = []
for cat, v in delta["Δ pp 2025–2024"].items():
    if pd.isna(v):
        lines_delta.append(f"{cat}: Δ 2025–2024 = n/a")
    else:
        lines_delta.append(f"{cat}: Δ 2025–2024 = {v:+.1f} pp")

print("\n".join(lines_delta))


Möbel / Wohnaccessoires: Δ 2025–2024 = -15.0 pp
Lebensmittel / Getränke: Δ 2025–2024 = -0.6 pp
Bücher / Medien / Software: Δ 2025–2024 = +21.2 pp
Elektronik (z. B. Smartphones, Haushaltsgeräte): Δ 2025–2024 = +5.6 pp
Medikamente / Drogerieartikel: Δ 2025–2024 = -22.8 pp
Hobby- & Freizeitartikel: Δ 2025–2024 = -3.2 pp
Kleidung / Schuhe: Δ 2025–2024 = -36.8 pp


In [7]:
# 06_heatmap_rows — Cell 5: Exporte
# TXT
txt_values = TAB / "06_heatmap_rows_values.txt"
txt_delta  = TAB / "06_heatmap_rows_delta_2025_minus_2024.txt"
txt_values.write_text("\n".join(lines_values), encoding="utf-8")
txt_delta.write_text("\n".join(lines_delta),  encoding="utf-8")
print("TXT exportiert →", txt_values)
print("TXT exportiert →", txt_delta)

# CSV (Werte)
csv_values = TAB / "06_heatmap_values.csv"
values.to_csv(csv_values, encoding="utf-8")
print("CSV exportiert →", csv_values)

# CSV (Δ)
csv_delta = TAB / "06_heatmap_delta_2025_minus_2024.csv"
delta.to_csv(csv_delta, encoding="utf-8")
print("CSV exportiert →", csv_delta)


TXT exportiert → D:\Q3_2025\data-analytics\project\reports\tables\06_heatmap_rows_values.txt
TXT exportiert → D:\Q3_2025\data-analytics\project\reports\tables\06_heatmap_rows_delta_2025_minus_2024.txt
CSV exportiert → D:\Q3_2025\data-analytics\project\reports\tables\06_heatmap_values.csv
CSV exportiert → D:\Q3_2025\data-analytics\project\reports\tables\06_heatmap_delta_2025_minus_2024.csv
